In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
from numpy import genfromtxt
import numpy as np
import csv
from collections import Counter

In [6]:
training_data = genfromtxt('training.csv',dtype = str, delimiter=',')
testing_data = genfromtxt('testing.csv',dtype = str, delimiter=',')

In [7]:
train_labels = training_data[1:,-1]
train_labels = (train_labels == 'b').astype(np.int32)
train_weights = training_data[1:,31].astype(np.float32)
train_data = training_data[1:,1:31]
train_data = train_data.astype(np.float32)

In [8]:
test_labels = testing_data[1:,-1]
test_labels = (test_labels == 'b').astype(np.int32)
test_weights = testing_data[1:,31].astype(np.float32)
test_data = testing_data[1:,1:31]
test_data = test_data.astype(np.float32)

In [9]:
def normalize_data(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    return data

def replace_missing_values(data):
    a, b = np.where(data == -999)
    new_r, new_c = np.where(data != -999)
    mean_list = np.mean(data[new_r][new_c], axis = 0)
    for i in range(len(a)):
        data[a[i]][data[a[i]] == -999] = mean_list[b[i]]
    x, y = np.where(data == -999)
    print(x,y)
    return data
    
def run_pca(data):
    pca = PCA(n_components=15)
    pca.fit(data)
    data = pca.transform(data)
    return data

In [10]:
train_data = replace_missing_values(train_data)
test_data = replace_missing_values(test_data)

train_data = normalize_data(train_data)
test_data = normalize_data(test_data)

train_data = run_pca(train_data)
test_data = run_pca(test_data)

print(train_data.shape)
print(test_data.shape)

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(250000, 15)
(618237, 15)


In [ ]:
print(Counter(train_data[:,22]))
data_jet_num_0 = train_data[train_data[:,22] == 0]
data_jet_num_0 = np.delete(data_jet_num_0,[4,5,6,12,22,23,24,25,26,27,28],1)
data_jet_num_0_labels = train_labels[np.where(train_data[:,22] == 0)]
print(Counter(data_jet_num_0_labels))
data_jet_num_1 = train_data[train_data[:,22] == 1]
data_jet_num_1 = np.delete(data_jet_num_1,[4,5,6,12,26,27,28,22],1)
data_jet_num_1_labels = train_labels[np.where(train_data[:,22] == 1)]
print(Counter(data_jet_num_1_labels))
data_jet_num_2 = train_data[train_data[:,22] == 2]
data_jet_num_2_labels = train_labels[np.where(train_data[:,22] == 2)]
print(Counter(data_jet_num_2_labels))
data_jet_num_3 = train_data[train_data[:,22] == 3]
data_jet_num_3_labels = train_labels[np.where(train_data[:,22] == 3)]
print(Counter(data_jet_num_3_labels))
data_jet_num_2_3 = np.concatenate((data_jet_num_2,data_jet_num_3))
data_jet_num_2_3 = np.delete(data_jet_num_2_3,[22],1)
data_jet_num_2_3_labels = np.concatenate((data_jet_num_2_labels,data_jet_num_3_labels))
print(Counter(data_jet_num_2_3_labels))
print(len(data_jet_num_0),len(data_jet_num_1),len(data_jet_num_2_3))

In [ ]:
scaler = StandardScaler()
scaler.fit(data_jet_num_0)
data_jet_num_0 = scaler.transform(data_jet_num_0)
scaler = StandardScaler()
scaler.fit(data_jet_num_1)
data_jet_num_1 = scaler.transform(data_jet_num_1)
scaler = StandardScaler()
scaler.fit(data_jet_num_2_3)
data_jet_num_2_3 = scaler.transform(data_jet_num_2_3)

In [ ]:
pca = PCA(0.99)
data_jet_num_0 = pca.fit_transform(data_jet_num_0)
print(pca.components_)
pca = PCA(0.99)
data_jet_num_1 = pca.fit_transform(data_jet_num_1)
print(pca.components_)
pca = PCA(0.99)
data_jet_num_2_3 = pca.fit_transform(data_jet_num_2_3)
print(pca.components_)

In [11]:
import math
def calc_ams(w,y,p):
    w = np.asarray(w)
    y = np.asarray(y)
    p = np.asarray(p)
    y_signal = w * (y == 0)
    y_background = w * (y == 1)
    s = np.sum(y_signal * (p == 1))
    b = np.sum(y_background * (p == 1))
    b_r=10.0
    a = np.sqrt( 2 * ((s + b + b_r) * math.log ( 1.0 + (s / (b + b_r) ) ) - s ))
    return a

In [12]:
from sklearn.metrics import precision_score, recall_score
def precision(labels, pred):
    precision = precision_score(labels,pred)
    return precision

def recall(labels, pred):
    recall = recall_score(labels,pred)
    return recall

In [13]:
from sklearn.linear_model import LogisticRegression
def run_lr(train_data,train_labels,test_data,test_labels,weights):
    clf = LogisticRegression(random_state=0, solver='lbfgs')
    clf = clf.fit(train_data, train_labels)
    preds = clf.predict(test_data)
    preds[preds<0.6] = 0
    preds[preds>=0.6] = 1
    print("Logistic Regression:")
    print("Accuracy: ", clf.score(test_data,test_labels))
    print("AMS score: ", calc_ams(weights.astype(np.float32),test_labels,preds))
    print("Precision: ", precision(test_labels,preds))
    print("Recall: ", recall(test_labels,preds))

In [14]:
from sklearn.naive_bayes import GaussianNB

def run_gnb(train_data,train_labels,test_data,test_labels,weights):
    clf = GaussianNB()
    clf = clf.fit(train_data, train_labels)
    preds = clf.predict(test_data)
    preds[preds<0.5] = 0
    preds[preds>=0.5] = 1
    print("Gaussian Naive Bayes:")
    print("Accuracy: ", clf.score(test_data,test_labels))
    print("AMS score: ", calc_ams(weights.astype(np.float32),test_labels,preds))
    print("Precision: ", precision(test_labels,preds))
    print("Recall: ", recall(test_labels,preds))

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

def run_gradient_boosting(train_data,train_labels,test_data,test_labels,weights):
    clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
    clf = clf.fit(train_data, train_labels)
    preds = clf.predict(test_data)
    preds[preds<0.5] = 0
    preds[preds>=0.5] = 1
    print("Gradient Boosting Classifier:")
    print("Accuracy: ", clf.score(test_data,test_labels))
    print("AMS score: ", calc_ams(weights.astype(np.float32),test_labels,preds))
    print("Precision: ", precision(test_labels,preds))
    print("Recall: ", recall(test_labels,preds))

In [16]:
from sklearn.tree import DecisionTreeClassifier

def run_decision_tree(train_data,train_labels,test_data,test_labels,weights):
    clf = DecisionTreeClassifier(random_state=0)
    clf = clf.fit(train_data, train_labels)
    preds = clf.predict(test_data)
    preds[preds<0.5] = 0
    preds[preds>=0.5] = 1
    print("Decision Tree Classifier:")
    print("Accuracy: ", clf.score(test_data,test_labels))
    print("AMS score: ", calc_ams(weights.astype(np.float32),test_labels,preds))
    print("Precision: ", precision(test_labels,preds))
    print("Recall: ", recall(test_labels,preds))

In [17]:
from xgboost import XGBClassifier
def run_xgboost(train_data,train_labels,test_data,test_labels,weights):
    clf = XGBClassifier()
    clf = clf.fit(train_data, train_labels)
    preds = clf.predict(test_data)
    preds[preds<0.5] = 0
    preds[preds>=0.5] = 1
    print("Decision Tree Classifier:")
    print("Accuracy: ", clf.score(test_data,test_labels))
    print("AMS score: ", calc_ams(weights.astype(np.float32),test_labels,preds))
    print("Precision: ", precision(test_labels,preds))
    print("Recall: ", recall(test_labels,preds))

In [13]:
import xgboost as xgb

def run_xgb(train_data,train_labels,test_data,test_labels,weights):
    label = train_labels
    y = weights
    weight = y * (float(len(test_labels)) / len(train_labels))

    sum_wpos = sum( weight[i] for i in range(len(label)) if label[i] == 1.0)
    sum_wneg = sum( weight[i] for i in range(len(label)) if label[i] == 0.0)
    print ('weight statistics: wpos=%g, wneg=%g, ratio=%g' % ( sum_wpos, sum_wneg, sum_wneg/sum_wpos ))

    xgmat = xgb.DMatrix( train_data, label=label, weight=weight )
    print(type(xgmat))
    # setup parameters for xgboost
    param = {}
    # use logistic regression loss, use raw prediction before logistic transformation
    # since we only need the rank
    param['objective'] = 'binary:logitraw'
    # scale weight of positive examples
    param['scale_pos_weight'] = sum_wneg/sum_wpos
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['eval_metric'] = 'auc'
    param['silent'] = 1
    param['nthread'] = 16

    # you can directly throw param in, though we want to watch multiple metrics here
    plst = list(param.items())+[('eval_metric', 'ams@0.15')]+[('eval_metric', 'error@0.3')]
    watchlist = [ (xgmat,'train') ]

    # boost 120 trees
    num_round = 120
    print ('loading data end, start to boost trees')

    bst = xgb.train(plst, xgmat, num_round, watchlist)
    preds = bst.predict(test_data)
    print(preds)
    # save out model
    bst.save_model('higgs.model')

In [ ]:
run_lr(train_data,train_labels,test_data,test_labels,test_weights)

In [ ]:
run_gnb(train_data,train_labels,test_data,test_labels,test_weights)

In [ ]:
run_gradient_boosting(train_data,train_labels,test_data,test_labels,test_weights)

In [ ]:
run_decision_tree(train_data,train_labels,test_data,test_labels,test_weights)

In [ ]:
run_xgboost(train_data,train_labels,test_data,test_labels,test_weights)

In [ ]:
# calculate Pearson's correlation
from scipy.stats import pearsonr
n = len(train_data[0])
dict1={}
for i in range(1,n-2):
    l= []
    for j in range(1,n-2):
        corr, _ = pearsonr(train_data[:,i], train_data[:,j])
        l.append(corr)
    dict1[training_data[0][i]]=l

dataframe = pd.DataFrame.from_dict(dict1, orient='index',columns=training_data[0][1:n-2])


#check for relations where there is 0.95 correlation
dict2={}
for key in dict1:
    n = len(dict1[key])
    for i in range(0,n):
        if dict1[key][i]>=0.95 and key!=training_data[0][i+1]:
            if key>training_data[0][i+1]:
                a = training_data[0][i+1]
                b = key
            else:
                a = key
                b = training_data[0][i+1]
            dict2[(a,b)]=dict1[key][i]

for key in dict2:
    print (key,dict2[key])

In [ ]:
dataCsv = dataframe.to_csv(index=True)
f = open('correlation.csv','w+')
f.write(dataCsv)

In [ ]:
# Plot of DER_mass_MC
import random
import numpy
from matplotlib import pyplot

filename='training.csv'
data = genfromtxt(filename,dtype = float, delimiter=',');
data1 = genfromtxt(filename,dtype = str, delimiter=',');
Mass = data[1:,1]
Y = data1[1:,32]
Y = np.array(Y)

Mass = np.array(Mass)
Msignal = Mass[Y=='s'];
Mbackground = Mass[Y=='b'];
bins = numpy.linspace(9, 500, 50)
pyplot.hist(Mbackground, bins, alpha=0.5, label='Background')
pyplot.hist(Msignal, bins, alpha=0.5, label='Signal')
pyplot.legend(loc='upper right')
pyplot.xlabel('DER_mass_MMC')
pyplot.ylabel('Number of events')
pyplot.show()

In [ ]:
from sklearn.svm import SVC
clf = SVC(probability=True)
clf.fit(train_data[:200000], train_labels[:200000])
clf.score(train_data[200000:], train_labels[200000:])

In [ ]:
#experimenting on different parameters of svm
from sklearn import svm
X,Y = train_data,train_labels
W = training_data[1:,31]
W = W.astype(np.float64)

X1 = X[:100000];
W1 = W[:100000];
Y1 = Y[:100000];
Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.001, 0.01, 0.1, 1]
kernels = ['linear', 'rbf', 'poly']

print("\nExperimenting for different kernels")
for kernel in kernels:
    svc= svm.SVC(kernel=kernel).fit(X1,Y1)
#     plotSVC('kernel=' + str(kernel))
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
    print("Kernel ", kernel," Svm score for test data:",svmScore)
    
print("\nExperimenting for different gammas")
for gamma in gammas:
    svc = svm.SVC(kernel='rbf', gamma=gamma).fit(X, Y)
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
#     plotSVC(‘gamma=’ + str(gamma))
    print("Gamma:", gamma," Svm score for test data:",svmScore)
    
print("\nExperimenting for different Regularization parameters")
for c in Cs:
    svc = svm.SVC(kernel='rbf', C=c).fit(X, Y)
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
#     plotSVC(‘C=’ + str(c))
    print("C:", c," Svm score for test data:",svmScore)

print("\nExperimenting for different Degrees for polynomial kernel")
degrees = [0, 1, 2, 3, 4]
for degree in degrees:
    svc = svm.SVC(kernel='poly', degree=degree).fit(X, Y)
    svmScore = svc.score(X[100000:200000],Y[100000:200000])
    print("Degree ", degree," Svm score for test data:",svmScore)
    